# Identifying Singular Matrices using Gaussian Elimination

In linear algebra, a **singular matrix** (or degenerate matrix) is a square matrix that does not have an inverse. This property is fundamental in many areas of mathematics, physics, and computer science. A matrix is singular if and only if its **determinant** is zero.

While calculating the determinant is a direct way to check for singularity, another powerful method is to attempt to convert the matrix into **row echelon form** through Gaussian elimination. If this process fails because a zero appears on the main diagonal that cannot be eliminated by row swapping or addition, the matrix is singular.

This notebook demonstrates a Python function that implements this test for a 4x4 matrix. 💡

## The Algorithm: Row Echelon Form

The core idea is to transform the matrix step-by-step, row by row, to achieve an upper triangular form (a form of row echelon form).

For each row `i` (from 0 to 3):
1.  **Zero Out Sub-diagonals**: Use linear combinations of previous rows (which already have a `1` on their diagonal) to make all elements below the main diagonal in the current column zero. For example, for `Row 2` (`A[2]`), we make `A[2,0]` and `A[2,1]` zero.
2.  **Normalize the Diagonal**: Ensure the diagonal element `A[i,i]` is not zero. If it is, we try to add a lower row to it to make it non-zero. If we can't (because all lower rows also have a zero in that column), the matrix is singular.
3.  **Set Diagonal to One**: Divide the entire row `i` by the value of its diagonal element `A[i,i]` to make the diagonal element equal to 1. 

If we can complete this process for all rows, the matrix is non-singular. If at any point we encounter a diagonal element that is zero and cannot be fixed, we raise an exception, flagging the matrix as singular.

## Python Implementation

First, we'll import `numpy` for efficient matrix operations and define a custom exception to handle the case of a singular matrix gracefully.

In [1]:
import numpy as np

# A custom exception to signal when the matrix is singular.
class MatrixIsSingular(Exception): pass

Next, we define the helper functions that process each row according to the algorithm described above.

In [2]:
def fixRowZero(A) :
    """Processes the first row (row 0)."""
    # If the first element is 0, add a lower row to it.
    if A[0,0] == 0 :
        A[0] = A[0] + A[1]
    if A[0,0] == 0 :
        A[0] = A[0] + A[2]
    if A[0,0] == 0 :
        A[0] = A[0] + A[3]
    # If it's still 0, the matrix is singular.
    if A[0,0] == 0 :
        raise MatrixIsSingular()
    # Normalize the row to make the diagonal element 1.
    A[0] = A[0] / A[0,0]
    return A

def fixRowOne(A) :
    """Processes the second row (row 1)."""
    # Zero out the sub-diagonal element A[1,0].
    A[1] = A[1] - (A[1,0] * A[0])
    
    # If the diagonal element is 0, add a lower row.
    if A[1,1] == 0 :
        A[1] = A[1] + A[2]
        A[1] = A[1] - A[1,0] * A[0]
    if A[1,1] == 0 :
        A[1] = A[1] + A[3]
        A[1] = A[1] - A[1,0] * A[0]
    # If it's still 0, the matrix is singular.
    if A[1,1] == 0 :
        raise MatrixIsSingular()
    # Normalize the row.
    A[1] = A[1] / A[1,1]
    return A

def fixRowTwo(A) :
    """Processes the third row (row 2)."""
    # Zero out the sub-diagonal elements A[2,0] and A[2,1].
    A[2] = A[2] - (A[2,0] * A[0])
    A[2] = A[2] - (A[2,1] * A[1])
    
    # If the diagonal element is 0, add the last row.
    if A[2,2] == 0 :
        A[2] = A[2] + A[3]
        # Repeat the zeroing out process.
        A[2] = A[2] - (A[2,0] * A[0])
        A[2] = A[2] - (A[2,1] * A[1])
    
    # If it's still 0, the matrix is singular.
    if A[2,2] == 0 :
        raise MatrixIsSingular()
    # Normalize the row.
    A[2] = A[2] / A[2,2]
    return A

def fixRowThree(A) :
    """Processes the final row (row 3)."""
    # Zero out the sub-diagonal elements A[3,0], A[3,1], and A[3,2].
    A[3] = A[3] - (A[3,0] * A[0])
    A[3] = A[3] - (A[3,1] * A[1])
    A[3] = A[3] - (A[3,2] * A[2])
    
    # There are no lower rows to add, so if the diagonal is 0, it's singular.
    if A[3,3] == 0:
        raise MatrixIsSingular()
    # Normalize the row.
    A[3] = A[3] / A[3,3]
    return A

Finally, the main function `isSingular` orchestrates the process by calling each `fixRow` function in order. It uses a `try...except` block to catch the `MatrixIsSingular` exception, returning `True` if it's caught and `False` otherwise.

In [3]:
def isSingular(A) :
    """ 
    Tests if a 4x4 matrix is singular by converting it to row echelon form.
    
    Args:
        A (numpy.ndarray): The 4x4 matrix to test.
    
    Returns:
        bool: True if the matrix is singular, False otherwise.
    """
    B = np.array(A, dtype=np.float64) # Make a copy to avoid modifying the original matrix.
    try:
        fixRowZero(B)
        fixRowOne(B)
        fixRowTwo(B)
        fixRowThree(B)
    except MatrixIsSingular:
        return True
    return False

## Demonstration

Let's test our function with a few examples.

### Case 1: A Singular Matrix

This matrix is singular because the last two rows are linearly dependent (Row 4 is 1.25 times Row 3). Our function should correctly identify this.

In [4]:
A_singular = np.array([
        [2, 0, 0, 0],
        [0, 3, 0, 0],
        [0, 0, 4, 4],
        [0, 0, 5, 5]
    ], dtype=np.float64)

print(f"Is the matrix singular? {isSingular(A_singular)}")

Is the matrix singular? True


### Case 2: A Non-Singular Matrix

This matrix looks more complex, but it is non-singular and has an inverse. Our function should return `False`.

In [5]:
A_nonsingular = np.array([
        [0, 7, -5, 3],
        [2, 8, 0, 4],
        [3, 12, 0, 5],
        [1, 3, 1, 3]
    ], dtype=np.float64)

print(f"Is the matrix singular? {isSingular(A_nonsingular)}")

Is the matrix singular? False


### Visualizing the Echelon Form Transformation

To see the process in action, let's apply the `fixRow` functions one by one to the non-singular matrix and observe how it's transformed into an upper-triangular matrix with ones on the diagonal.

In [6]:
M = np.array([
        [0, 7, -5, 3],
        [2, 8, 0, 4],
        [3, 12, 0, 5],
        [1, 3, 1, 3]
    ], dtype=np.float64)

print("Original Matrix:\n", M)

fixRowZero(M)
print("\nAfter fixRowZero:\n", M)

fixRowOne(M)
print("\nAfter fixRowOne:\n", M)

fixRowTwo(M)
print("\nAfter fixRowTwo:\n", M)

fixRowThree(M)
print("\nAfter fixRowThree (Final Echelon Form):\n", M)

Original Matrix:
 [[ 0.  7. -5.  3.]
 [ 2.  8.  0.  4.]
 [ 3. 12.  0.  5.]
 [ 1.  3.  1.  3.]]

After fixRowZero:
 [[ 1.   7.5 -2.5  3.5]
 [ 2.   8.   0.   4. ]
 [ 3.  12.   0.   5. ]
 [ 1.   3.   1.   3. ]]

After fixRowOne:
 [[ 1.          7.5        -2.5         3.5       ]
 [-0.          1.         -0.71428571  0.42857143]
 [ 3.         12.          0.          5.        ]
 [ 1.          3.          1.          3.        ]]

After fixRowTwo:
 [[ 1.          7.5        -2.5         3.5       ]
 [-0.          1.         -0.71428571  0.42857143]
 [ 0.          0.          1.          1.5       ]
 [ 1.          3.          1.          3.        ]]

After fixRowThree (Final Echelon Form):
 [[ 1.          7.5        -2.5         3.5       ]
 [-0.          1.         -0.71428571  0.42857143]
 [ 0.          0.          1.          1.5       ]
 [ 0.          0.          0.          1.        ]]


## Conclusion

This manual implementation of Gaussian elimination provides a clear understanding of the mechanics behind identifying singular matrices. By converting a matrix to row echelon form, we can determine its invertibility. While `numpy.linalg.det(A) == 0` is a more direct and computationally optimized method, building the process from scratch is a fantastic exercise for reinforcing key concepts in linear algebra. ✅